# Python-1, Лекция 14

Лектор: Хайбулин Даниэль

Подготовил материал: Хайбулин Даниэль

## Сравнение тестовых фреймворков Python

| Фреймворк  | Плюсы                                                                 | Минусы                                                                 |
|------------|-----------------------------------------------------------------------|------------------------------------------------------------------------|
| **unittest** | • Входит в стандартную библиотеку<br>• JUnit-стиль (знаком многим)<br>• Поддержка фикстур (setUp/tearDown) | • Много boilerplate-кода<br>• Менее читаемые ассерты<br>• Синтаксис сложнее, чем у современных аналогов |
| **pytest**   | • Минимальный boilerplate<br>• Читаемые ассерты (`assert x == y`)<br>• Параметризация тестов<br>• Плагины и богатая экосистема<br>• Поддержка unittest-тестов | • Требует установки (не в стандартной библиотеке)<br>• Синтаксис фикстур может быть сложным для новичков |
| **doctest**  | • Встроен в стандартную библиотеку<br>• Тесты как документация<br>• Нет лишнего кода | • Не подходит для сложных тестов<br>• Сложность отладки<br>• Может нарушать читаемость документации |
| **nose2**    | • Совместимость с unittest<br>• Упрощенный запуск тестов<br>• Плагины | • Развитие замедлилось после появления pytest<br>• Меньше возможностей, чем у pytest |

**Рекомендации**:
- **pytest** — лучший выбор для большинства проектов

## Именование тестов

Как вообще утилита **pytest** находит тесты среди прочих файлов?

1. Находит все файлы, которые оканчиваются на *.py
2. Оставляет только файлы вида `test_*.py`, `*_test.py`
3. Внутри файлов:
    * находит все функции с префиксом `test`;
    * находит все методы с префиксом `test` внутри класса с прекфиксом `Test`.

[конвенция](https://docs.pytest.org/en/stable/explanation/goodpractices.html#test-discovery)

## TDD

**TDD** - Test Driven Development: сперва пишем тест для необходимой логики, затем пишем код для этой логики, основываясь на нашем тесте.

In [ ]:
%pip install pytest ipytest

In [ ]:
import pytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest


def test_good_math():
    assert 2 + 2 == 4


def test_bad_math():
    assert 2 + 2 == 5

`pytest` выводит отчет, в котором можно посмотреть сколько у нас всего тестов, какие из них упали и по какой причине.

Три правила TDD:

1. Продакшн-код можно писать только для починки падающего теста.

2. В тесте нужно писать ровно столько кода, сколько необходимо, чтобы он упал. Ошибки компиляции считаются падениями теста.

3. В продакшн можно написать ровно столько кода, сколько требуется для починки одного падающего теста.

Получается следуйющий пайплайн: пишем падающий тест, пишем код, чтобы тест не падал, рефакторим код так, чтобы тесты не падали. Повторяем до сходимости.

1. [Test Driven Development: By Example 1st Edition](https://www.amazon.com/Test-Driven-Development-Kent-Beck/dp/0321146530)

2. [On Growing Object Oriented Software, Guided by Tests](https://www.amazon.com/Growing-Object-Oriented-Software-Guided-Tests/dp/0321503627)

## Ката

Каты — упражнения по программированию, помогающие отточить навыки путем многократного повторнения. Концепция взята из японских боевых искусств. Подробнее про них можно почитать в книжке [The Pragmatic Programmer](https://pragprog.com/titles/tpp20/the-pragmatic-programmer-20th-anniversary-edition/)

### Ката Greeter

Эту кату надо выполнять строго по пунктам, не заглядывая вперёд.

* Создайте класс `Greeter`, у которого есть метод `greet` принимающий на вход имя и возвращающий "Hello <имя>".

Давайте сперва напишем тест, в котором проверяем конструктор класса:

In [ ]:
%%ipytest


def test_greeter():
    Greeter()

Теперь нам нужно починить тест минимальным количеством кода:

In [ ]:
class Greeter: ...

Далее сделаем еще одну итерацию написания теста:

In [ ]:
%%ipytest


def test_greeter_2():
    Greeter().greet("Daniel")

In [ ]:
class Greeter:  # noqa: F811
    def greet(self, name: str) -> None:
        return f"Hello, {name}!"

В конце концов напишем финальный тест, который что-то проверяет. Тут нам поможет знакомый нам `assert`:

In [ ]:
%%ipytest


def test_greeter_3():
    name = "Daniel"
    assert Greeter().greet(name=name) == f"Hello, {name}!"

## Параметризация

Давайте теперь напишем несколько тестов, проверяющих наш метод:

In [ ]:
%%ipytest


def test_greeter_name():
    name = "Daniel"
    assert Greeter().greet(name=name) == f"Hello, {name}!"


def test_greeter_empty():
    name = ""
    assert Greeter().greet(name=name) == f"Hello, {name}!"

Чем больше кейсов, которые мы хотим проверить, тем больше у нас тестирующих функций - это не очень удобно. Давайте посмотрим как можно делать несколько разных кейсов в одном тесте. С этим нам поможет *параметризация*:

In [ ]:
%%ipytest
test_cases = [
    ("Daniel", "Hello, Daniel!"),
    ("", "Hello, !"),
]


@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter_parametrized(name, greeting):
    assert Greeter().greet(name) == greeting

Следующий пункт нашей каты: метод `greet` должен обрезать пробельные символы в по краям имени.

In [ ]:
%%ipytest
test_cases = [("Daniel", "Hello, Daniel!"), ("", "Hello, !"), ("\n\t\n\t", "Hello, !")]


@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter_with_trim(name, greeting):
    assert Greeter().greet(name) == greeting

Пора опять чинить код, чтобы он проходил наш тест:

In [ ]:
class Greeter:  # noqa: F811
    def greet(self, name: str) -> None:
        return f"Hello, {name.strip()}!"

In [ ]:
%%ipytest
test_cases = [("Daniel", "Hello, Daniel!"), ("", "Hello, !"), ("\n\t\n\t", "Hello, !")]


@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter_with_trim(name, greeting):  # noqa: F811
    assert Greeter().greet(name) == greeting

## Фикстуры

Фикстуры - очень сильный механизм, помогающий создать общий контекст для тестов.

In [ ]:
%%ipytest


class Fruit:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name


@pytest.fixture
def my_fruit():
    return Fruit("apple")


@pytest.fixture
def fruit_basket(my_fruit):
    return [Fruit("banana"), my_fruit]


def test_my_fruit_in_basket(my_fruit, fruit_basket):
    assert my_fruit in fruit_basket

В нашем случае мы для каждого тест кейса создаем объект класса `Greeter`, что нам не особо нужно. Давайте попробуем решить эту проблему с помощью фикстур:

In [ ]:
%%ipytest

from typing import Iterator

test_cases = [("Daniel", "Hello, Daniel!"), ("", "Hello, !"), ("\n\t\n\t", "Hello, !")]


@pytest.fixture(scope="module")
def greeter() -> Iterator[Greeter]:
    yield Greeter()


@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter_with_trim(greeter: Greeter, name: str, greeting: str):  # noqa: F811
    assert greeter.greet(name) == greeting

Фикстуры могут иметь разный скоуп - как для целового модуля тестов, так и для отдельного теста. Фикстуры часто используются как контекстные менеджеры: ведь удобно в тесте подключиться к чему-то, а после теста закрыть это дело. Простой пример из жизни - подключение к базе данных в тесте, оно должно быть закрыто после выполнения теста.

In [ ]:
%%ipytest -s


class DBConnection:
    pass


class TestDB:
    def init_db(self):
        print("init db")

    def get_connection(self) -> DBConnection:
        return DBConnection()

    def shutdown(self):
        print("close db")


@pytest.fixture(scope="module")
def db_connection() -> Iterator[DBConnection]:
    db = TestDB()
    db.init_db()
    try:
        yield db.get_connection()
    finally:
        db.shutdown()


def test_db_1(db_connection: DBConnection):
    assert db_connection


def test_db_2(db_connection: DBConnection):
    assert db_connection

Тест не ограничивается одной фикстурой в аргументе: можно прописывать сколько угодно фикстур для тестов.

In [ ]:
%%ipytest

import pytest


@pytest.fixture
def first_entry():
    return "a"


@pytest.fixture
def second_entry():
    return 2


@pytest.fixture
def order(first_entry, second_entry):
    return [first_entry, second_entry]


@pytest.fixture
def expected_list():
    return ["a", 2, 3.0]


def test_string(order, expected_list):
    order.append(3.0)

    assert order == expected_list

Фикстура может автоматически использоваться для каждого теста.

In [ ]:
%%ipytest

import pytest


@pytest.fixture
def first_entry():  # noqa: F811
    return "a"


@pytest.fixture
def order(first_entry):  # noqa: F811
    return []


@pytest.fixture(autouse=True)
def append_first(order, first_entry):
    return order.append(first_entry)


def test_string_only(order, first_entry):
    assert order == [first_entry]


def test_string_and_int(order, first_entry):
    order.append(2)
    assert order == [first_entry, 2]

## Тестирование исключений

Следующий пункт нашей каты: метод `greet` должен возвращать исключение если передана пустая строка (или строка, состоящая только из пробельных символов).

Для тестирования исключений есть специальный контекстный менеджер в модуле `pytest`.

In [ ]:
%%ipytest

from typing import Iterator

test_cases = [("Daniel", "Hello, Daniel!"), ("", "Hello, !"), ("\n\t\n\t", "Hello, !")]


@pytest.fixture(scope="module")
def greeter() -> Iterator[Greeter]:  # noqa: F811
    yield Greeter()


@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter_with_trim(greeter: Greeter, name: str, greeting: str):  # noqa: F811
    assert greeter.greet(name) == greeting

## Сравнение float

In [ ]:
%%ipytest


def test_float():
    assert 0.1 + 0.2 == 0.3

In [ ]:
%%ipytest


def test_float():  # noqa: F811
    assert 0.1 + 0.2 == pytest.approx(0.3)

In [ ]:
%%ipytest


def test_float():  # noqa: F811
    assert [0.1 + 0.2, 0.5] == pytest.approx([0.3, 0.5])

## Попрактикуемся

Нам нужно покрыть следующий модуль тестами:

In [ ]:
class MailAdminClient:
    def create_user(self) -> "MailUser":
        return MailUser()

    def delete_user(self, user: "MailUser") -> None:
        pass


class MailUser:
    def __init__(self):
        self.inbox = []

    def send_email(self, email: "Email", other: "MailUser") -> None:
        other.inbox.append(email)

    def clear_mailbox(self) -> None:
        self.inbox.clear()


class Email:
    def __init__(self, subject: str, body: str):
        self.subject = subject
        self.body = body

In [ ]:
%%ipytest

import pytest
from typing import Iterator


@pytest.fixture
def mail_admin():
    return MailAdminClient()


@pytest.fixture
def sending_user(mail_admin) -> Iterator[MailUser]:
    user = mail_admin.create_user()
    yield user
    mail_admin.delete_user(user)


@pytest.fixture
def receiving_user(mail_admin) -> Iterator[MailUser]:
    user = mail_admin.create_user()
    yield user
    user.clear_mailbox()
    mail_admin.delete_user(user)


def test_email_received(sending_user, receiving_user):
    email = Email(subject="Hey!", body="How's it going?")
    sending_user.send_email(email, receiving_user)
    assert email in receiving_user.inbox